In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"
    
execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb;reload(pu_delayfb)
import matplotlib as mpl
mpl.style.use('seaborn-bright')
%matplotlib inline

def savefig(name):
    plt.savefig(name + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(name + ".pdf", bbox_inches='tight')

In [ ]:
reload(pu_delayfb)

N_sweep_msmts_ots = {
    'N2': '20170615161800',
    'N4': '20170615162800',
    'N6': '20170615163300'
}

Ns = np.array([2, 4, 6])

det_sweep_x = []
det_sweep_y = []
det_sweep_y_u = []
det_sweep_fr = []

for N in Ns:
    sfr, sx, sy, sy_u = pu_delayfb.calibrate_LDE_phase(contains = 'phase_fb_delayline',
                                older_than = N_sweep_msmts_ots['N%d' % N],
                                do_fit = True,fixed = [1],show_guess = True, plot_fit=False, 
                                freq = 6/360.,decay = 100,phi0=0.0, ret_fit_data=True
                                #offset=0.5, A0 = 0.4
                               )
    det_sweep_x.append(sx)
    det_sweep_y.append(sy)
    det_sweep_y_u.append(sy_u)
    det_sweep_fr.append(sfr)

In [ ]:
reload(pu_delayfb)

fig = plt.figure()
ax = plt.subplot(111)

xN2, yN2, _ = pu_delayfb.analyse_delay_feedback_phase_error('fb_delayline', older_than=N_sweep_msmts_ots['N2'], name='X_positive', label='N = 2', ret_data=True, all_carbons_saved=False)
xN4, yN4, _ = pu_delayfb.analyse_delay_feedback_phase_error('fb_delayline', older_than=N_sweep_msmts_ots['N4'], name='X_positive', label='N = 4', ret_data=True, all_carbons_saved=False)
xN6, yN6, _ = pu_delayfb.analyse_delay_feedback_phase_error('fb_delayline', older_than=N_sweep_msmts_ots['N6'], name='X_positive', label='N = 6', ret_data=True, all_carbons_saved=False)

plt.legend(loc=3)

plt.savefig("phase_error_spread_LDE2_attempts.png", dpi=300)

In [ ]:
phase_error_mean = np.zeros((3))
phase_error_std = np.zeros((3))

phase_error_arrays = [yN2, yN4, yN6]

for i, arr in enumerate(phase_error_arrays):
    phase_error_mean[i] = np.mean(arr)
    phase_error_std[i] = np.std(arr)
    
plt.figure()
# plt.scatter(Ns, phase_error_mean, label='mean')
plt.scatter(Ns, phase_error_std, label='std dev')
plt.xlim(0,6.5)
plt.ylim(0,12)

plt.xlabel(r"Number of feedback pulses $N$")
plt.ylabel(r"Std. dev. of the phase error $\sigma_{\Delta\phi}$")

plt.savefig("std_dev_phase_error_delay_N.png", dpi=300)

In [ ]:
def fit_decaying_cos_with_phase_errors(g_f, g_a, g_A, g_phi,g_t,phase_errors):
    fitfunc_str = 'A *exp(-x/t) cos(2pi * (f*x + (phi + phi_err/360) ) + a'

    f = fit.Parameter(g_f, 'f')
    a = fit.Parameter(g_a, 'a')
    A = fit.Parameter(g_A, 'A')
    phi = fit.Parameter(g_phi, 'phi')
    t   = fit.Parameter(g_t, 't')
    # print 'guessed frequency is '+str(g_f)
    p0 = [f, a, A,phi,t]

    def fitfunc(x):
        return a() + A()*np.exp(-x/t()) * np.cos(2*np.pi*( f()*x + (phi()+phase_errors)/360.))

    return p0, fitfunc, fitfunc_str

def plot_func_decaying_cos_with_phase_errors(x, p0, phase_errors):
    f, a, A,phi,t = p0
    
    return a + A*np.exp(-x/t) * np.cos(2*np.pi*( f*x + (phi+phase_errors)/360.))

In [ ]:
reload(pu_delayfb)

lns = []

plt.figure(figsize=(4,3))

for i in [0, 2]:
    N = Ns[i]
    sx, sy, sy_u, sfr = det_sweep_x[i], det_sweep_y[i], det_sweep_y_u[i], det_sweep_fr[i]

#     fig = plt.figure()
    if i == 0:
        ax = plt.subplot(111)
    else:
        ax2 = ax.twinx()
    plt.xlabel("Number of LDE attempts")
    if i == 0:
        plt.ylabel((r"$\langle X \rangle$" % N) + "\t\t")
    else:
        plt.ylabel("\t\t"+r"$\langle X \rangle$" % N)

    xvals = np.linspace(0., 60., 1000.)

    msmt_eb = plt.errorbar(sx, sy, sy_u, fmt='o', label="N = %d" % N, color="C%d" % i)

    fit_ln = plt.plot(xvals, sfr['fitfunc'](xvals), color="C%d" % i, label="N = %d" % N)
    fit_plus_phase_errors = np.zeros_like(phase_error_arrays[i])

    old_phi = sfr['params_dict']['phi']
    phi_idx = 2

    for j, phase_err in enumerate(phase_error_arrays[i]):
        sfr['p0'][phi_idx].set(old_phi + phase_err)
        fit_plus_phase_errors[j] = sfr['fitfunc'](sx[j])
    sfr['p0'][phi_idx].set(old_phi)
    pherr_eb = plt.errorbar(sx, fit_plus_phase_errors, sy_u, markersize=8, fmt='*', label="N = %d fit plus phase error" % N, color="C%d" % ((3*i)/2+1))
    
    lns += [msmt_eb]
    
dummy_eb = plt.errorbar(np.array([]), np.array([]), markersize=8, fmt='*', color='k', label="fit plus phase error")
lns += [dummy_eb]

yticks = [-0.5, 0.0, 0.5]
ax.set_ylim(-0.5, 1.0)
ax.set_yticks(yticks)
ax.tick_params(axis='y', colors='C0')
ax.yaxis.label.set_color("C0")
ax.spines['left'].set_color("C0")
ax.spines['right'].set_color("C2")

ax2.set_ylim(-1.0,0.5)
ax2.set_yticks(yticks)
ax2.tick_params(axis='y', colors='C2')
ax2.yaxis.label.set_color("C2")
ax2.spines['left'].set_color("C0")
ax2.spines['right'].set_color("C2")

labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc=9)

plt.xlim(0,60)

savefig("thesis-plots/phase-resolution-errors")

#     plt.savefig("detuning_sweep_normal_fit_with_phase_errors_N_%d.png" % N, dpi=300)

In [ ]:
msmt_eb[2][0]

In [ ]:
reload(pu_delayfb)

i = 0
N = Ns[i]

sx, sy, sy_u = det_sweep_x[i], det_sweep_y[i], det_sweep_y_u[i]



freq = 6.0/360.0
phi0 = 0.0
decay = 100.0
offset = 0.0
A0 = 0.5

fixed = [1]

p0, fitfunc, fitfunc_str = fit_decaying_cos_with_phase_errors(freq,offset,A0,phi0,decay, phase_errors=phase_error_arrays[i])
sfr = fit.fit1d(sx, sy,None,p0=p0, fitfunc=fitfunc, do_print=True, ret=True, VERBOSE=True, fixed=fixed)


fig = plt.figure()
ax = plt.subplot(111)
plt.xlabel("Number of LDE attempts")
plt.ylabel(r"$\langle X \rangle$")



xvals = np.linspace(0, np.max(sx))

color = plt.errorbar(sx, sy, sy_u, fmt='o', label="N = %d" % N)[0].get_color()

params_dict = sfr['params_dict']
p0 = [params_dict['f'], offset, params_dict['A'], params_dict['phi'], params_dict['t']]

plotfunc = lambda x: plot_func_decaying_cos_with_phase_errors(x, p0, 0.0)

plt.plot(xvals, plotfunc(xvals), label="N = %d, fit, phase errors corrected" % N)
fit_plus_phase_errors = np.zeros_like(phase_error_arrays[i])

fit_plus_phase_errors = plot_func_decaying_cos_with_phase_errors(sx, p0, phase_errors=phase_error_arrays[i])

plt.errorbar(sx, fit_plus_phase_errors, sy_u, fmt='x', label="N = %d, fit with phase errors" % N)

plt.legend()

plt.savefig("detuning_sweep_phase_error_fit_N_%d.png" % N, dpi=300)

In [ ]:
reload(pu_delayfb)

multiC_ot = "20170710174200"

contains = "phase_fb_delayline"

### acquire data
f = pu_delayfb.toolbox.latest_data(contains, older_than=multiC_ot)
a = pu_delayfb.PurificationDelayFBAnalysis(f)

In [ ]:
name = "X_positive"
max_nuclei = 6

agrp = a.adwingrp(name)
reps = agrp.attrs['reps_per_ROsequence']
pts = agrp.attrs['sweep_length']
sweep_pts = agrp.attrs['sweep_pts']
sweep_name = agrp.attrs['sweep_name']

feedback_delay_cycles = agrp['feedback_delay_cycles'].value.reshape((max_nuclei, pts, reps), order='F')

In [ ]:
feedback_delay_cycles[2]